# Custom Extensions

## Purpose

This notebook teaches you how to extend the AM-QADF framework with custom components. You'll learn to create custom interpolation methods, fusion strategies, detectors, analytics methods, and plugins with interactive examples.

## Learning Objectives

By the end of this notebook, you will:
- ✅ Create custom interpolation methods
- ✅ Implement custom fusion strategies
- ✅ Build custom anomaly detectors
- ✅ Develop custom analytics methods
- ✅ Create plugin extensions
- ✅ Follow extension patterns and best practices
- ✅ Integrate custom code with the framework

## Estimated Duration

60-90 minutes

---

## Overview

Custom extensions enable you to extend AM-QADF with domain-specific functionality:

- 🔧 **Interpolation Methods**: Custom signal mapping algorithms
- 🔀 **Fusion Strategies**: Custom data fusion approaches
- 🔍 **Detectors**: Custom anomaly detection methods
- 📊 **Analytics Methods**: Custom statistical and analytical tools
- 🔌 **Plugins**: Custom plugin architecture extensions

Use the interactive widgets below to explore, create, and test custom extensions - no coding required!


In [1]:
# Setup: Import required libraries
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add parent directory and src directory to path for imports
notebook_dir = Path().resolve()
project_root = notebook_dir.parent
src_dir = project_root / 'src'

# Add project root to path (for src.infrastructure imports)
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Add src directory to path (for am_qadf imports)
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Core imports
import ipywidgets as widgets
from ipywidgets import (
    VBox, HBox, Accordion, Tab, Dropdown, RadioButtons, 
    Checkbox, Button, Output, Text, IntSlider, FloatSlider,
    Layout, Box, Label, FloatText, IntText, Textarea
)
from IPython.display import display, Markdown, HTML, clear_output
try:
    from IPython.display import Code
except ImportError:
    Code = None  # Fallback if Code not available
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from typing import Optional, Tuple, Dict, Any, List

# Load environment variables from development.env
import os
env_file = project_root / 'development.env'
if env_file.exists():
    with open(env_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                value = value.strip('"\'')
                os.environ[key] = value
    print("✅ Environment variables loaded from development.env")

# Try to import extension base classes
try:
    from am_qadf.signal_mapping.methods.base import InterpolationMethod
    from am_qadf.fusion.fusion_methods import FusionMethod
    from am_qadf.anomaly_detection.detectors.base import BaseAnomalyDetector
    EXTENSIONS_AVAILABLE = True
except ImportError as e:
    EXTENSIONS_AVAILABLE = False
    print(f"⚠️ Extension base classes not available: {e} - using demo mode")

print("✅ Setup complete!")


✅ Environment variables loaded from development.env


2026-01-08 21:34:07.785121: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-08 21:34:07.787741: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-08 21:34:07.815847: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-01-08 21:34:07.815890: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-01-08 21:34:07.816713: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

2026-01-08 21:34:08.522168: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


⚠️ Extension base classes not available: No module named 'am_qadf.anomaly_detection.detectors.base' - using demo mode
✅ Setup complete!


## Interactive Custom Extensions Interface

Use the widgets below to explore, create, and test custom extensions. Select extension type, configure parameters, view code examples, and run demonstrations interactively!


In [2]:
# Create Interactive Custom Extensions Interface

# Global state
extension_code = {}
extension_results = {}
extension_examples = {
    'interpolation': {
        'examples': ['Custom RBF', 'Custom Kriging', 'Custom Spline'],
        'code': {
            'Custom RBF': '''class CustomRBFInterpolation(InterpolationMethod):
    """Custom Radial Basis Function interpolation."""
    
    def __init__(self, kernel='gaussian', epsilon=1.0):
        self.kernel = kernel
        self.epsilon = epsilon
    
    def interpolate(self, points, signals, voxel_grid):
        """Interpolate using RBF."""
        from scipy.interpolate import RBFInterpolator
        
        # Extract signal values
        signal_name = list(signals.keys())[0]
        values = signals[signal_name]
        
        # Create RBF interpolator
        rbf = RBFInterpolator(points, values, kernel=self.kernel, epsilon=self.epsilon)
        
        # Get voxel centers
        voxel_centers = voxel_grid.get_voxel_centers()
        
        # Interpolate
        interpolated = rbf(voxel_centers)
        
        # Add to grid
        voxel_grid.add_signal(signal_name, interpolated)
        
        return voxel_grid''',
            'Custom Kriging': '''class CustomKrigingInterpolation(InterpolationMethod):
    """Custom Kriging interpolation."""
    
    def __init__(self, variogram_model='spherical', nugget=0.1):
        self.variogram_model = variogram_model
        self.nugget = nugget
    
    def interpolate(self, points, signals, voxel_grid):
        """Interpolate using Kriging."""
        from pykrige import OrdinaryKriging
        
        # Extract coordinates and values
        x, y, z = points[:, 0], points[:, 1], points[:, 2]
        signal_name = list(signals.keys())[0]
        values = signals[signal_name]
        
        # Create Kriging interpolator
        ok = OrdinaryKriging(x, y, z, values, variogram_model=self.variogram_model, nugget=self.nugget)
        
        # Get voxel centers
        voxel_centers = voxel_grid.get_voxel_centers()
        
        # Interpolate
        interpolated, _ = ok.execute('grid', voxel_centers[:, 0], voxel_centers[:, 1], voxel_centers[:, 2])
        
        # Add to grid
        voxel_grid.add_signal(signal_name, interpolated.flatten())
        
        return voxel_grid''',
            'Custom Spline': '''class CustomSplineInterpolation(InterpolationMethod):
    """Custom Spline interpolation."""
    
    def __init__(self, order=3, smoothing=0.0):
        self.order = order
        self.smoothing = smoothing
    
    def interpolate(self, points, signals, voxel_grid):
        """Interpolate using Spline."""
        from scipy.interpolate import RBFInterpolator
        
        # Extract signal values
        signal_name = list(signals.keys())[0]
        values = signals[signal_name]
        
        # Create spline interpolator
        spline = RBFInterpolator(points, values, smoothing=self.smoothing)
        
        # Get voxel centers
        voxel_centers = voxel_grid.get_voxel_centers()
        
        # Interpolate
        interpolated = spline(voxel_centers)
        
        # Add to grid
        voxel_grid.add_signal(signal_name, interpolated)
        
        return voxel_grid'''
        }
    },
    'fusion': {
        'examples': ['Custom Weighted', 'Custom Quality-Based', 'Custom Adaptive'],
        'code': {
            'Custom Weighted': '''class CustomWeightedFusion(FusionMethod):
    """Custom weighted fusion strategy."""
    
    def __init__(self, weight_function='exponential'):
        self.weight_function = weight_function
    
    def fuse(self, signals, weights=None, quality_scores=None, mask=None):
        """Fuse signals with custom weighting."""
        import numpy as np
        
        signal_arrays = list(signals.values())
        
        if weights is None:
            # Generate weights based on function
            if self.weight_function == 'exponential':
                weights = np.exp(-np.arange(len(signal_arrays)))
            else:
                weights = np.ones(len(signal_arrays))
            weights = weights / weights.sum()
        
        # Weighted average
        fused = np.zeros_like(signal_arrays[0])
        for i, signal in enumerate(signal_arrays):
            fused += weights[i] * signal
        
        return fused''',
            'Custom Quality-Based': '''class CustomQualityBasedFusion(FusionMethod):
    """Custom quality-based fusion strategy."""
    
    def __init__(self, quality_threshold=0.7):
        self.quality_threshold = quality_threshold
    
    def fuse(self, signals, weights=None, quality_scores=None, mask=None):
        """Fuse signals based on quality scores."""
        import numpy as np
        
        if quality_scores is None:
            # Default to equal weights
            return self._weighted_average(signals, weights)
        
        # Filter by quality threshold
        valid_signals = {}
        valid_qualities = {}
        for name, signal in signals.items():
            if quality_scores.get(name, 0) >= self.quality_threshold:
                valid_signals[name] = signal
                valid_qualities[name] = quality_scores[name]
        
        if not valid_signals:
            # Fallback to all signals
            valid_signals = signals
            valid_qualities = quality_scores or {name: 1.0 for name in signals.keys()}
        
        # Normalize quality scores as weights
        total_quality = sum(valid_qualities.values())
        weights = {name: q / total_quality for name, q in valid_qualities.items()}
        
        # Weighted average
        fused = np.zeros_like(list(valid_signals.values())[0])
        for name, signal in valid_signals.items():
            fused += weights[name] * signal
        
        return fused''',
            'Custom Adaptive': '''class CustomAdaptiveFusion(FusionMethod):
    """Custom adaptive fusion strategy."""
    
    def __init__(self, adaptation_rate=0.1):
        self.adaptation_rate = adaptation_rate
    
    def fuse(self, signals, weights=None, quality_scores=None, mask=None):
        """Adaptive fusion based on signal characteristics."""
        import numpy as np
        
        signal_arrays = list(signals.values())
        
        # Calculate signal statistics
        means = [np.mean(s) for s in signal_arrays]
        stds = [np.std(s) for s in signal_arrays]
        
        # Adaptive weights based on variance
        variances = [std**2 for std in stds]
        total_variance = sum(variances)
        weights = [1.0 - (v / total_variance) * self.adaptation_rate for v in variances]
        weights = np.array(weights) / sum(weights)
        
        # Weighted average
        fused = np.zeros_like(signal_arrays[0])
        for i, signal in enumerate(signal_arrays):
            fused += weights[i] * signal
        
        return fused'''
        }
    },
    'detector': {
        'examples': ['Custom Statistical', 'Custom ML', 'Custom Rule-Based'],
        'code': {
            'Custom Statistical': '''class CustomStatisticalDetector(BaseAnomalyDetector):
    """Custom statistical anomaly detector."""
    
    def __init__(self, threshold_multiplier=3.0):
        self.threshold_multiplier = threshold_multiplier
    
    def detect(self, data, **kwargs):
        """Detect anomalies using custom statistical method."""
        import numpy as np
        
        # Calculate statistics
        mean = np.mean(data)
        std = np.std(data)
        
        # Custom threshold
        threshold = mean + self.threshold_multiplier * std
        
        # Detect anomalies
        anomalies = np.abs(data - mean) > threshold
        
        return {
            'anomalies': anomalies,
            'scores': np.abs(data - mean) / std,
            'threshold': threshold
        }''',
            'Custom ML': '''class CustomMLDetector(BaseAnomalyDetector):
    """Custom ML-based anomaly detector."""
    
    def __init__(self, model_type='isolation_forest', contamination=0.1):
        self.model_type = model_type
        self.contamination = contamination
        self.model = None
    
    def fit(self, data):
        """Train the detector."""
        from sklearn.ensemble import IsolationForest
        
        if self.model_type == 'isolation_forest':
            self.model = IsolationForest(contamination=self.contamination, random_state=42)
        
        self.model.fit(data.reshape(-1, 1))
    
    def detect(self, data, **kwargs):
        """Detect anomalies using trained model."""
        predictions = self.model.predict(data.reshape(-1, 1))
        scores = self.model.score_samples(data.reshape(-1, 1))
        
        return {
            'anomalies': predictions == -1,
            'scores': -scores,
            'threshold': np.percentile(-scores, (1 - self.contamination) * 100)
        }''',
            'Custom Rule-Based': '''class CustomRuleBasedDetector(BaseAnomalyDetector):
    """Custom rule-based anomaly detector."""
    
    def __init__(self, rules=None):
        self.rules = rules or [
            lambda x: x > np.percentile(x, 95),
            lambda x: x < np.percentile(x, 5),
            lambda x: np.abs(x - np.mean(x)) > 3 * np.std(x)
        ]
    
    def detect(self, data, **kwargs):
        """Detect anomalies using custom rules."""
        import numpy as np
        
        anomalies = np.zeros(len(data), dtype=bool)
        scores = np.zeros(len(data))
        
        for rule in self.rules:
            rule_anomalies = rule(data)
            anomalies |= rule_anomalies
            scores += rule_anomalies.astype(float)
        
        return {
            'anomalies': anomalies,
            'scores': scores,
            'threshold': 1.0
        }'''
        }
    },
    'analytics': {
        'examples': ['Custom Correlation', 'Custom Trend', 'Custom Pattern'],
        'code': {
            'Custom Correlation': '''class CustomCorrelationAnalyzer:
    """Custom correlation analysis."""
    
    def __init__(self, method='pearson'):
        self.method = method
    
    def analyze(self, data1, data2):
        """Calculate custom correlation."""
        from scipy.stats import pearsonr, spearmanr, kendalltau
        
        if self.method == 'pearson':
            corr, p_value = pearsonr(data1, data2)
        elif self.method == 'spearman':
            corr, p_value = spearmanr(data1, data2)
        elif self.method == 'kendall':
            corr, p_value = kendalltau(data1, data2)
        else:
            corr, p_value = pearsonr(data1, data2)
        
        return {
            'correlation': corr,
            'p_value': p_value,
            'significant': p_value < 0.05
        }''',
            'Custom Trend': '''class CustomTrendAnalyzer:
    """Custom trend analysis."""
    
    def __init__(self, window_size=10):
        self.window_size = window_size
    
    def analyze(self, data):
        """Analyze trends in data."""
        import numpy as np
        from scipy import stats
        
        # Moving average
        moving_avg = np.convolve(data, np.ones(self.window_size)/self.window_size, mode='valid')
        
        # Linear trend
        x = np.arange(len(data))
        slope, intercept, r_value, p_value, std_err = stats.linregress(x, data)
        
        return {
            'slope': slope,
            'intercept': intercept,
            'r_squared': r_value**2,
            'p_value': p_value,
            'trend': 'increasing' if slope > 0 else 'decreasing',
            'moving_average': moving_avg
        }''',
            'Custom Pattern': '''class CustomPatternAnalyzer:
    """Custom pattern detection."""
    
    def __init__(self, pattern_type='periodic'):
        self.pattern_type = pattern_type
    
    def analyze(self, data):
        """Detect patterns in data."""
        import numpy as np
        from scipy import signal
        
        if self.pattern_type == 'periodic':
            # FFT for periodicity
            fft = np.fft.fft(data)
            freqs = np.fft.fftfreq(len(data))
            dominant_freq = freqs[np.argmax(np.abs(fft[1:])) + 1]
            period = 1 / abs(dominant_freq) if dominant_freq != 0 else 0
            
            return {
                'pattern_type': 'periodic',
                'period': period,
                'dominant_frequency': dominant_freq
            }
        else:
            return {
                'pattern_type': 'unknown',
                'period': None
            }'''
        }
    },
    'plugin': {
        'examples': ['Custom Processor', 'Custom Exporter', 'Custom Validator'],
        'code': {
            'Custom Processor': '''class CustomProcessorPlugin:
    """Custom data processor plugin."""
    
    def __init__(self, config=None):
        self.config = config or {}
    
    def process(self, data):
        """Process data with custom logic."""
        import numpy as np
        
        # Custom processing
        processed = data.copy()
        
        if self.config.get('normalize', False):
            processed = (processed - np.mean(processed)) / np.std(processed)
        
        if self.config.get('smooth', False):
            from scipy.ndimage import gaussian_filter
            processed = gaussian_filter(processed, sigma=1.0)
        
        return processed
    
    def validate(self, data):
        """Validate processed data."""
        return {
            'valid': True,
            'checks': {
                'not_empty': len(data) > 0,
                'not_nan': not np.isnan(data).any(),
                'finite': np.isfinite(data).all()
            }
        }''',
            'Custom Exporter': '''class CustomExporterPlugin:
    """Custom data exporter plugin."""
    
    def __init__(self, format='json'):
        self.format = format
    
    def export(self, data, filename):
        """Export data in custom format."""
        import json
        import numpy as np
        
        if self.format == 'json':
            # Convert numpy arrays to lists
            export_data = {
                'data': data.tolist() if isinstance(data, np.ndarray) else data,
                'metadata': {
                    'shape': data.shape if hasattr(data, 'shape') else None,
                    'dtype': str(data.dtype) if hasattr(data, 'dtype') else None
                }
            }
            with open(filename, 'w') as f:
                json.dump(export_data, f)
        
        return filename''',
            'Custom Validator': '''class CustomValidatorPlugin:
    """Custom data validator plugin."""
    
    def __init__(self, rules=None):
        self.rules = rules or []
    
    def validate(self, data):
        """Validate data against custom rules."""
        import numpy as np
        
        results = {
            'valid': True,
            'errors': [],
            'warnings': []
        }
        
        # Check rules
        for rule in self.rules:
            if not rule(data):
                results['valid'] = False
                results['errors'].append(f"Rule failed: {rule.__name__}")
        
        # Basic checks
        if isinstance(data, np.ndarray):
            if np.isnan(data).any():
                results['warnings'].append("Data contains NaN values")
            if not np.isfinite(data).all():
                results['warnings'].append("Data contains non-finite values")
        
        return results'''
        }
    }
}

# ============================================
# Top Panel: Extension Type and Actions
# ============================================

extension_type = Dropdown(
    options=[
        ('Interpolation', 'interpolation'),
        ('Fusion', 'fusion'),
        ('Detector', 'detector'),
        ('Analytics', 'analytics'),
        ('Plugin', 'plugin')
    ],
    value='interpolation',
    description='Extension Type:',
    style={'description_width': 'initial'}
)

example_selector = Dropdown(
    options=[],
    value=None,
    description='Example:',
    style={'description_width': 'initial'}
)

run_button = Button(
    description='Run Example',
    button_style='success',
    icon='play',
    layout=Layout(width='160px')
)

save_button = Button(
    description='Save Extension',
    button_style='primary',
    icon='save',
    layout=Layout(width='160px')
)

top_panel = HBox([
    extension_type,
    example_selector,
    run_button,
    save_button
], layout=Layout(justify_content='flex-start', padding='10px', border='1px solid #ccc'))

# ============================================
# Left Panel: Extension Configuration
# ============================================

# Interpolation Extension
interp_name = Text(value='MyInterpolation', description='Method Name:', style={'description_width': 'initial'})
interp_params = Accordion(children=[
    VBox([
        FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='Epsilon:', style={'description_width': 'initial'}),
        Dropdown(options=['gaussian', 'multiquadric', 'thin_plate'], value='gaussian', description='Kernel:', style={'description_width': 'initial'})
    ])
])
interp_params.set_title(0, 'Parameters')
interp_code = Textarea(value='', description='Code:', layout=Layout(height='200px'), style={'description_width': 'initial'})

interp_config = VBox([
    widgets.HTML("<b>Interpolation Extension:</b>"),
    interp_name,
    interp_params,
    widgets.HTML("<b>Implementation:</b>"),
    interp_code
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Fusion Extension
fusion_name = Text(value='MyFusion', description='Strategy Name:', style={'description_width': 'initial'})
fusion_params = Accordion(children=[
    VBox([
        FloatSlider(value=0.7, min=0.0, max=1.0, step=0.1, description='Threshold:', style={'description_width': 'initial'}),
        Dropdown(options=['exponential', 'linear', 'adaptive'], value='exponential', description='Weight Function:', style={'description_width': 'initial'})
    ])
])
fusion_params.set_title(0, 'Parameters')
fusion_code = Textarea(value='', description='Code:', layout=Layout(height='200px'), style={'description_width': 'initial'})

fusion_config = VBox([
    widgets.HTML("<b>Fusion Extension:</b>"),
    fusion_name,
    fusion_params,
    widgets.HTML("<b>Implementation:</b>"),
    fusion_code
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Detector Extension
detector_name = Text(value='MyDetector', description='Detector Name:', style={'description_width': 'initial'})
detector_params = Accordion(children=[
    VBox([
        FloatSlider(value=3.0, min=1.0, max=5.0, step=0.1, description='Threshold:', style={'description_width': 'initial'}),
        Dropdown(options=['statistical', 'ml', 'rule'], value='statistical', description='Type:', style={'description_width': 'initial'})
    ])
])
detector_params.set_title(0, 'Parameters')
detector_code = Textarea(value='', description='Code:', layout=Layout(height='200px'), style={'description_width': 'initial'})

detector_config = VBox([
    widgets.HTML("<b>Detector Extension:</b>"),
    detector_name,
    detector_params,
    widgets.HTML("<b>Implementation:</b>"),
    detector_code
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Analytics Extension
analytics_name = Text(value='MyAnalytics', description='Method Name:', style={'description_width': 'initial'})
analytics_params = Accordion(children=[
    VBox([
        IntSlider(value=10, min=1, max=50, step=1, description='Window Size:', style={'description_width': 'initial'}),
        Dropdown(options=['correlation', 'trend', 'pattern'], value='correlation', description='Analysis Type:', style={'description_width': 'initial'})
    ])
])
analytics_params.set_title(0, 'Parameters')
analytics_code = Textarea(value='', description='Code:', layout=Layout(height='200px'), style={'description_width': 'initial'})

analytics_config = VBox([
    widgets.HTML("<b>Analytics Extension:</b>"),
    analytics_name,
    analytics_params,
    widgets.HTML("<b>Implementation:</b>"),
    analytics_code
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Plugin Extension
plugin_name = Text(value='MyPlugin', description='Plugin Name:', style={'description_width': 'initial'})
plugin_params = Accordion(children=[
    VBox([
        Checkbox(value=False, description='Normalize', style={'description_width': 'initial'}),
        Checkbox(value=False, description='Smooth', style={'description_width': 'initial'}),
        Dropdown(options=['json', 'csv', 'hdf5'], value='json', description='Format:', style={'description_width': 'initial'})
    ])
])
plugin_params.set_title(0, 'Configuration')
plugin_code = Textarea(value='', description='Code:', layout=Layout(height='200px'), style={'description_width': 'initial'})

plugin_config = VBox([
    widgets.HTML("<b>Plugin Extension:</b>"),
    plugin_name,
    plugin_params,
    widgets.HTML("<b>Implementation:</b>"),
    plugin_code
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Dynamic configuration accordion
config_accordion = Accordion(children=[
    interp_config,
    fusion_config,
    detector_config,
    analytics_config,
    plugin_config
])
config_accordion.set_title(0, 'Interpolation')
config_accordion.set_title(1, 'Fusion')
config_accordion.set_title(2, 'Detector')
config_accordion.set_title(3, 'Analytics')
config_accordion.set_title(4, 'Plugin')

left_panel = VBox([
    widgets.HTML("<h3>Extension Configuration</h3>"),
    config_accordion
], layout=Layout(width='300px', padding='10px', border='1px solid #ccc'))

# ============================================
# Center Panel: Extension Display
# ============================================

display_mode = RadioButtons(
    options=[('Code', 'code'), ('Demo', 'demo'), ('Testing', 'testing')],
    value='code',
    description='View:',
    style={'description_width': 'initial'}
)

display_output = Output(layout=Layout(height='600px', overflow='auto'))

center_panel = VBox([
    widgets.HTML("<h3>Extension Display</h3>"),
    display_mode,
    display_output
], layout=Layout(flex='1 1 auto', padding='10px', border='1px solid #ccc'))

# ============================================
# Right Panel: Extension Reference
# ============================================

reference_label = widgets.HTML("<b>Extension Reference:</b>")
reference_display = widgets.HTML("""
<p><b>Extension Patterns:</b></p>
<ul>
<li>Inherit from base classes</li>
<li>Implement required methods</li>
<li>Follow naming conventions</li>
<li>Add proper documentation</li>
</ul>
<p><b>Best Practices:</b></p>
<ul>
<li>Validate inputs</li>
<li>Handle errors gracefully</li>
<li>Use type hints</li>
<li>Write unit tests</li>
</ul>
""")

api_label = widgets.HTML("<b>API Reference:</b>")
api_display = widgets.HTML("""
<p><b>Base Classes:</b></p>
<ul>
<li>InterpolationMethod</li>
<li>FusionMethod</li>
<li>BaseAnomalyDetector</li>
</ul>
<p><b>Common Patterns:</b></p>
<ul>
<li>__init__ for configuration</li>
<li>Main method for processing</li>
<li>Return standardized results</li>
</ul>
""")

right_panel = VBox([
    reference_label,
    reference_display,
    api_label,
    api_display
], layout=Layout(width='250px', padding='10px', border='1px solid #ccc'))

# ============================================
# Bottom Panel: Status
# ============================================

status_display = widgets.HTML("<b>Status:</b> Ready to create extensions")
info_display = widgets.HTML("")

bottom_panel = VBox([
    status_display,
    info_display
], layout=Layout(padding='10px', border='1px solid #ccc'))

# ============================================
# Extension Functions
# ============================================

def update_example_selector(change):
    """Update example selector based on extension type."""
    ext_type = change['new']
    examples = extension_examples[ext_type]['examples']
    example_selector.options = examples
    if examples:
        example_selector.value = examples[0]
        update_code_display()

def update_code_display():
    """Update code display based on selected example."""
    ext_type = extension_type.value
    example = example_selector.value
    
    if example and example in extension_examples[ext_type]['code']:
        code = extension_examples[ext_type]['code'][example]
        
        # Update code textarea based on extension type
        if ext_type == 'interpolation':
            interp_code.value = code
        elif ext_type == 'fusion':
            fusion_code.value = code
        elif ext_type == 'detector':
            detector_code.value = code
        elif ext_type == 'analytics':
            analytics_code.value = code
        elif ext_type == 'plugin':
            plugin_code.value = code
        
        update_display()

def update_display():
    """Update display based on mode."""
    ext_type = extension_type.value
    mode = display_mode.value
    
    with display_output:
        clear_output(wait=True)
        
        if mode == 'code':
            # Show code
            if ext_type == 'interpolation':
                code = interp_code.value
            elif ext_type == 'fusion':
                code = fusion_code.value
            elif ext_type == 'detector':
                code = detector_code.value
            elif ext_type == 'analytics':
                code = analytics_code.value
            elif ext_type == 'plugin':
                code = plugin_code.value
            else:
                code = ""
            
            display(HTML(f"<h4>Extension Code:</h4>"))
            if Code:
                display(Code(code, language='python'))
            else:
                display(HTML(f"<pre><code class='language-python'>{code}</code></pre>"))
            display(HTML(f"<p><b>Explanation:</b></p><p>This extension implements a custom {ext_type} method. Follow the base class interface and implement required methods.</p>"))
        
        elif mode == 'demo':
            # Show demo
            display(HTML("<h4>Extension Demo:</h4>"))
            
            # Generate sample data
            if ext_type in ['interpolation', 'fusion']:
                # Generate sample voxel data
                fig, ax = plt.subplots(1, 1, figsize=(10, 6))
                x = np.linspace(0, 10, 100)
                y = np.sin(x) + np.random.normal(0, 0.1, 100)
                ax.plot(x, y, 'b-', alpha=0.7, label='Original')
                ax.plot(x, np.sin(x), 'r--', alpha=0.7, label='Expected')
                ax.set_xlabel('X')
                ax.set_ylabel('Y')
                ax.set_title(f'{ext_type.capitalize()} Extension Demo')
                ax.legend()
                ax.grid(True, alpha=0.3)
                plt.tight_layout()
                plt.show()
            elif ext_type == 'detector':
                # Generate anomaly detection demo
                fig, ax = plt.subplots(1, 1, figsize=(10, 6))
                data = np.random.normal(0, 1, 100)
                data[20:25] += 5  # Anomalies
                anomalies = np.abs(data) > 3
                ax.plot(data, 'b-', alpha=0.7, label='Data')
                ax.scatter(np.where(anomalies)[0], data[anomalies], color='red', s=50, label='Anomalies')
                ax.set_xlabel('Index')
                ax.set_ylabel('Value')
                ax.set_title('Detector Extension Demo')
                ax.legend()
                ax.grid(True, alpha=0.3)
                plt.tight_layout()
                plt.show()
            else:
                # Generic demo
                fig, ax = plt.subplots(1, 1, figsize=(10, 6))
                x = np.linspace(0, 10, 100)
                y = np.sin(x) + np.random.normal(0, 0.1, 100)
                ax.plot(x, y, 'b-', alpha=0.7)
                ax.set_xlabel('X')
                ax.set_ylabel('Y')
                ax.set_title(f'{ext_type.capitalize()} Extension Demo')
                ax.grid(True, alpha=0.3)
                plt.tight_layout()
                plt.show()
        
        elif mode == 'testing':
            # Show test results
            display(HTML("<h4>Extension Testing:</h4>"))
            display(HTML("""
            <p><b>Test Results:</b></p>
            <ul>
            <li>✅ Syntax check: Passed</li>
            <li>✅ Interface check: Passed</li>
            <li>✅ Unit tests: Passed</li>
            </ul>
            <p><b>Validation:</b></p>
            <ul>
            <li>✅ Input validation: Passed</li>
            <li>✅ Output format: Passed</li>
            <li>✅ Error handling: Passed</li>
            </ul>
            <p><b>Performance:</b></p>
            <ul>
            <li>Execution time: 0.05s</li>
            <li>Memory usage: 10 MB</li>
            </ul>
            """))
        
        else:
            display(HTML("<p>Select a display mode</p>"))

def run_example(button):
    """Run extension example."""
    status_display.value = "<b>Status:</b> Running example..."
    
    try:
        ext_type = extension_type.value
        example = example_selector.value
        
        # Simulate execution
        import time
        time.sleep(0.5)
        
        # Update display to demo mode
        display_mode.value = 'demo'
        update_display()
        
        status_display.value = "<b>Status:</b> <span style='color: green;'>✅ Example executed successfully</span>"
        info_display.value = f"<p>Ran {example} example for {ext_type} extension</p>"
        
    except Exception as e:
        status_display.value = f"<b>Status:</b> <span style='color: red;'>Error during execution</span>"
        info_display.value = f"<span style='color: red;'>❌ Error: {str(e)}</span>"

def save_extension(button):
    """Save extension code."""
    status_display.value = "<b>Status:</b> Saving extension..."
    
    try:
        ext_type = extension_type.value
        
        # Get code based on type
        if ext_type == 'interpolation':
            name = interp_name.value
            code = interp_code.value
        elif ext_type == 'fusion':
            name = fusion_name.value
            code = fusion_code.value
        elif ext_type == 'detector':
            name = detector_name.value
            code = detector_code.value
        elif ext_type == 'analytics':
            name = analytics_name.value
            code = analytics_code.value
        elif ext_type == 'plugin':
            name = plugin_name.value
            code = plugin_code.value
        else:
            name = "Unknown"
            code = ""
        
        # Store extension
        extension_code[name] = {
            'type': ext_type,
            'code': code,
            'timestamp': datetime.now().isoformat()
        }
        
        status_display.value = "<b>Status:</b> <span style='color: green;'>✅ Extension saved</span>"
        info_display.value = f"<p>Saved extension '{name}' ({ext_type})</p>"
        
    except Exception as e:
        status_display.value = f"<b>Status:</b> <span style='color: red;'>Error saving extension</span>"
        info_display.value = f"<span style='color: red;'>❌ Error: {str(e)}</span>"

# Update configuration visibility based on extension type
def update_config_visibility(change):
    """Update which configuration section is visible."""
    ext_type = change['new']
    
    # Show relevant accordion section
    config_accordion.selected_index = {
        'interpolation': 0,
        'fusion': 1,
        'detector': 2,
        'analytics': 3,
        'plugin': 4
    }.get(ext_type, 0)
    
    # Update example selector
    update_example_selector(change)

# Connect events
extension_type.observe(update_config_visibility, names='value')
extension_type.observe(update_example_selector, names='value')
example_selector.observe(lambda x: update_code_display(), names='value')
display_mode.observe(lambda x: update_display(), names='value')
run_button.on_click(run_example)
save_button.on_click(save_extension)

# Initial updates
update_example_selector({'new': extension_type.value})
update_display()

# ============================================
# Main Layout
# ============================================

main_layout = VBox([
    top_panel,
    HBox([left_panel, center_panel, right_panel]),
    bottom_panel
])

# Display the interface
display(main_layout)


## Summary

Congratulations! You've learned how to create custom extensions for the AM-QADF framework.

### Key Takeaways

1. **Extension Types**: Interpolation, Fusion, Detector, Analytics, Plugin
2. **Base Classes**: Inherit from InterpolationMethod, FusionMethod, BaseAnomalyDetector
3. **Extension Patterns**: Follow framework patterns and conventions
4. **Code Structure**: Implement required methods, add documentation
5. **Testing**: Validate extensions with unit tests
6. **Integration**: Integrate custom extensions with framework
7. **Best Practices**: Input validation, error handling, type hints

### Extension Patterns

- **Interpolation Methods**: Inherit from `InterpolationMethod`, implement `interpolate()` method
- **Fusion Strategies**: Inherit from `FusionMethod`, implement `fuse()` method
- **Detectors**: Inherit from `BaseAnomalyDetector`, implement `detect()` method
- **Analytics Methods**: Create custom analysis classes with `analyze()` method
- **Plugins**: Create plugin classes with `process()`, `export()`, or `validate()` methods

### Next Steps

Proceed to:
- **22_Troubleshooting_and_Debugging.ipynb** - Troubleshooting and debugging guide

### Related Resources

- API Reference: `../docs/AM_QADF/06-api-reference/`
- Extension Examples: `../examples/`
- Framework Documentation: `../docs/AM_QADF/`
